# Transactions

In [6]:
import itertools
from datetime import timedelta
import numbers
from datetime import datetime

class TimeZone:
    def __init__(self, name, offset_hours, offset_minutes):
        if name is None or len(str(name).strip()) == 0:
            raise ValueError('Timezone name cannot be empty.')
        
        self._name = str(name).strip()
        
        if not isinstance(offset_hours, numbers.Integral):
            raise ValueError('Hour offset must be an integer.')
        
        if not isinstance(offset_minutes, numbers.Integral):
            raise ValueError('Minute offset must be an integer.')
        
        if offset_minutes > 59 or offset_minutes < -59:
            raise ValueError('Minutes offset must be between -59 and 59 (inclusive).')
        
        offset = timedelta(hours=offset_hours, minutes=offset_minutes)
        if offset < timedelta(hours=-12, minutes=0) or offset > timedelta(hours=14, minutes=0):
            raise ValueError('Offset must be between -12:00 and +14:00.')
        
        self._offset_hours = offset_hours
        self._offset_minutes = offset_minutes
        self._offset = offset
        
    @property
    def offset(self):
        return self._offset
    
    @property
    def name(self):
        return self._name
    
    # @name.setter
    # def name(self, value):
    #     self._name = value
    
    def __eq__(self, other):
        return (isinstance(other, TimeZone) and
                self.name == other.name and
                self._offset_hours == other._offset_hours and
                self._offset_minutes == other._offset_minutes)
    
    def __repr__(self):
        return (f"TimeZone(name='{self.name}',"
                f"offset_hours={self._offset_hours},"
                f"offset_minutes={self._offset_minutes})")

In [7]:
from collections import namedtuple

Confirmation = namedtuple('Confirmation', 'account_number transaction_code\
                            transaction_id time_utc time')

In [9]:
class Account:
    transaction_id = itertools.count(100)
    _interest_rate = 0.5 # percent
    
    _transaction_codes = {
        'deposit': 'D',
        'withdraw': 'W',
        'interest': 'I',
        'rejected': 'X'
    }
    
    def __init__(self, account_number, first_name, last_name,
                 initial_balance=0,timezone=None):
        self._account_number = account_number
        self.first_name = first_name
        self.last_name = last_name
        
        if timezone is None:
            timezone = TimeZone('UTC', 0, 0)
        self.timezone = timezone
        
        # in reality we should we decimal instead of float
        self._balance = float(initial_balance)
        
    @property
    def account_number(self):
        return self._account_number
    
    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        # if len(str(value).strip()) == 0:
        #     raise ValueError('First name cannot be empty.')
        # self._first_name = value
        self.validate_and_set_name('_first_name', value, 'First Name')
    @property
    def last_name(self):
        return self._last_name
    
    @last_name.setter
    def last_name(self, value):
        # if len(str(value).strip()) == 0:
        #     raise ValueError('Last name cannot be empty')
        # self._last_name = value
        self.validate_and_set_name('_last_name', value, 'Last Name')
    
    @property
    def balance(self):
        return self._balance
    
    @property
    def timezone(self):
        return self._timezone
    
    @timezone.setter
    def timezone(self, value):
        if not isinstance(value, TimeZone):
            raise ValueError('Time Zone must be a valid TimeZone object.')
        self._timezone = value
    
    @classmethod
    def get_interest_rate(cls):
        # we could have also done it as
        # return Account._interest_rate
        return cls._interest_rate
    
    @classmethod
    def set_interest_rate(cls, value):
        if not isinstance(value, numbers.Real):
            raise ValueError('Interest rate must be a real number.')
        
        if value < 0:
            raise ValueError('Interest rate cannot be negative.')
        cls._interest_rate = value
    
    def generate_confirmation_code(self,transaction_code):
        dt_str = datetime.utcnow().strftime('%Y%m%d%H%M%S')
        return f'{transaction_code}-{self.account_number}-{dt_str}-{next(Account.transaction_id)}'


    def validate_and_set_name(self, attr_name, value, field_title):
        # if len(str(value).strip()) == 0:
        if value is None or len(str(value).strip()) == 0: 
            raise ValueError(f'{field_title} cannot be empty')
        setattr(self, attr_name, value)
    
    def make_transaction(self):
        return self.generate_confirmation_code('dummy')
    
    @staticmethod
    def parse_confirmation_code(confirmation_code, preferred_time_zone=None):
        # dummy-A100-20200102141212-102
        parts = confirmation_code.split('-')
        if not (len(parts) == 4):
            raise ValueError('Invalid confirmation code')
        
        # transaction_code = parts[0]
        transaction_code, account_number, raw_dt_utc, transaction_id = parts
        
        try:
            # datetime.strptime is a string parse time function
            dt_utc = datetime.strptime(raw_dt_utc, '%Y%m%d%H%M%S')
        except ValueError as ex:
            # from ex means keeping the stack trace thats already there before
            raise ValueError('Invalid transaction datetime.') from ex
        
        if preferred_time_zone is None:
            preferred_time_zone = TimeZone('UTC', 0, 0)
            
        if not isinstance(preferred_time_zone, TimeZone):
            raise ValueError('Invalid TimeZone specified.')
        
        dt_preferred = dt_utc + preferred_time_zone.offset
        dt_preferred_str = f'{dt_preferred.strftime("%Y-%m-%d %H:%M:%S")} ({preferred_time_zone.name})'
        
        return Confirmation(account_number, transaction_code,
                            transaction_id, dt_utc.isoformat(), dt_preferred_str)

    def deposit(self, value):
        if not isinstance(value, numbers.Real):
            raise ValueError('Deposit value must be a real number.')
        if value <= 0:
            raise ValueError('Deposit value must a positive number.')
        
        # use the class name
        transaction_code = Account._transaction_codes['deposit'] 
        conf_code = self.generate_confirmation_code(transaction_code)
        self._balance += value
        return conf_code
    
    def withdraw(self, value):
        # TODO: Refactor to use common validation here and in deposit method
        
        accepted = False
        if self.balance - value < 0:
            # insufficient funds
            transaction_code = Account._transaction_codes['rejected']
        else:
            accepted = True
            transaction_code = Account._transaction_codes['withdraw']    
        
        conf_code = self.generate_confirmation_code(transaction_code)
        
        if accepted:
            self._balance -= value
            
        return conf_code
    
    def pay_interest(self):
        interest = self.balance * Account.get_interest_rate() / 100
        conf_code = self.generate_confirmation_code(self._transaction_codes['interest'])
        self._balance += interest
        return conf_code

In [10]:
a = Account('A100', 'Eric', 'Idle', initial_balance=100)

In [11]:
a.balance

100.0

In [12]:
try:
    a.deposit(-100)
except ValueError as ex:
    print(ex)

Deposit value must a positive number.


In [13]:
a.deposit(100)

'D-A100-20200103150555-100'

In [14]:
a.balance

200.0

In [15]:
a.withdraw(150)

'W-A100-20200103150606-101'

In [16]:
a.balance

50.0

In [17]:
a.withdraw(100)

'X-A100-20200103150618-102'

In [18]:
a.balance

50.0

In [21]:
class Account:
    transaction_id = itertools.count(100)
    _interest_rate = 0.5 # percent
    
    _transaction_codes = {
        'deposit': 'D',
        'withdraw': 'W',
        'interest': 'I',
        'rejected': 'X'
    }
    
    def __init__(self, account_number, first_name, last_name,
                 initial_balance=0,timezone=None):
        self._account_number = account_number
        self.first_name = first_name
        self.last_name = last_name
        
        if timezone is None:
            timezone = TimeZone('UTC', 0, 0)
        self.timezone = timezone
        
        # in reality we should we decimal instead of float
        self._balance = float(initial_balance)
        
    @property
    def account_number(self):
        return self._account_number
    
    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        # if len(str(value).strip()) == 0:
        #     raise ValueError('First name cannot be empty.')
        # self._first_name = value
        self.validate_and_set_name('_first_name', value, 'First Name')
    @property
    def last_name(self):
        return self._last_name
    
    @last_name.setter
    def last_name(self, value):
        # if len(str(value).strip()) == 0:
        #     raise ValueError('Last name cannot be empty')
        # self._last_name = value
        self.validate_and_set_name('_last_name', value, 'Last Name')
    
    @property
    def balance(self):
        return self._balance
    
    @property
    def timezone(self):
        return self._timezone
    
    @timezone.setter
    def timezone(self, value):
        if not isinstance(value, TimeZone):
            raise ValueError('Time Zone must be a valid TimeZone object.')
        self._timezone = value
    
    @classmethod
    def get_interest_rate(cls):
        # we could have also done it as
        # return Account._interest_rate
        return cls._interest_rate
    
    @classmethod
    def set_interest_rate(cls, value):
        if not isinstance(value, numbers.Real):
            raise ValueError('Interest rate must be a real number.')
        
        if value < 0:
            raise ValueError('Interest rate cannot be negative.')
        cls._interest_rate = value
    
    @staticmethod
    def validate_real_number(value, min_value=None):
        if not isinstance(value, numbers.Real):
            raise ValueError('Value must be a real number.')
        
        if min_value is not None and value < min_value:
            raise ValueError(f'Value must be at least {min_value}.')
         
        return value
    
    def generate_confirmation_code(self,transaction_code):
        dt_str = datetime.utcnow().strftime('%Y%m%d%H%M%S')
        return f'{transaction_code}-{self.account_number}-{dt_str}-{next(Account.transaction_id)}'


    def validate_and_set_name(self, attr_name, value, field_title):
        # if len(str(value).strip()) == 0:
        if value is None or len(str(value).strip()) == 0: 
            raise ValueError(f'{field_title} cannot be empty')
        setattr(self, attr_name, value)
    
    def make_transaction(self):
        return self.generate_confirmation_code('dummy')
    
    @staticmethod
    def parse_confirmation_code(confirmation_code, preferred_time_zone=None):
        # dummy-A100-20200102141212-102
        parts = confirmation_code.split('-')
        if not (len(parts) == 4):
            raise ValueError('Invalid confirmation code')
        
        # transaction_code = parts[0]
        transaction_code, account_number, raw_dt_utc, transaction_id = parts
        
        try:
            # datetime.strptime is a string parse time function
            dt_utc = datetime.strptime(raw_dt_utc, '%Y%m%d%H%M%S')
        except ValueError as ex:
            # from ex means keeping the stack trace thats already there before
            raise ValueError('Invalid transaction datetime.') from ex
        
        if preferred_time_zone is None:
            preferred_time_zone = TimeZone('UTC', 0, 0)
            
        if not isinstance(preferred_time_zone, TimeZone):
            raise ValueError('Invalid TimeZone specified.')
        
        dt_preferred = dt_utc + preferred_time_zone.offset
        dt_preferred_str = f'{dt_preferred.strftime("%Y-%m-%d %H:%M:%S")} ({preferred_time_zone.name})'
        
        return Confirmation(account_number, transaction_code,
                            transaction_id, dt_utc.isoformat(), dt_preferred_str)

    def deposit(self, value):
        value = Account.validate_real_number(value, 0.01)
        # use the class name
        transaction_code = Account._transaction_codes['deposit'] 
        conf_code = self.generate_confirmation_code(transaction_code)
        self._balance += value
        return conf_code
    
    def withdraw(self, value):
        # TODO: Refactor to use common validation here and in deposit method
        value = Account.validate_real_number(value, 0.01)
        accepted = False
        if self.balance - value < 0:
            # insufficient funds
            transaction_code = Account._transaction_codes['rejected']
        else:
            accepted = True
            transaction_code = Account._transaction_codes['withdraw']    
        
        conf_code = self.generate_confirmation_code(transaction_code)
        
        if accepted:
            self._balance -= value
            
        return conf_code
    
    def pay_interest(self):
        interest = self.balance * Account.get_interest_rate() / 100
        conf_code = self.generate_confirmation_code(self._transaction_codes['interest'])
        self._balance += interest
        return conf_code



In [22]:
a = Account('A100', 'Eric', 'Idle', initial_balance=100)

In [23]:
a.balance

100.0

In [24]:
try:
    a.deposit(-100)
except ValueError as ex:
    print(ex)

Value must be at least 0.01.


In [25]:
a.deposit(100)

'D-A100-20200103151731-100'

In [26]:
try:
    a.withdraw(-100)
except ValueError as ex:
    print(ex)



Value must be at least 0.01.


In [27]:
try:
    a.withdraw(20)
except ValueError as ex:
    print(ex)




In [28]:
a.balance

180.0

In [29]:
a.deposit(100)

'D-A100-20200103151837-102'

In [30]:
a.balance

280.0

In [31]:
a.withdraw(150)

'W-A100-20200103151853-103'

In [32]:
a.balance

130.0

In [33]:
a.withdraw(100)

'W-A100-20200103151904-104'

In [34]:
a.withdraw(100)



'X-A100-20200103151920-105'